In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,46268
2,Huelva,Confirmados PDIA 14 días,391
3,Huelva,Tasa PDIA 14 días,"76,19307441978292"
4,Huelva,Confirmados PDIA 7 días,198
5,Huelva,Tasa PDIA 7 dias,"38,583705204902856"
6,Huelva,Total Confirmados,46472
7,Huelva,Curados,45171
8,Huelva,Fallecidos,420


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  46268.0


/tmp/ipykernel_65577/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12715.0


/tmp/ipykernel_65577/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_65577/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_65577/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_65577/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2438 personas en los últimos 7 días 

Un positivo PCR cada 1498 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,46268.0,198.0,391.0,513170.0,38.583705,76.193074,60.0
Huelva-Costa,27365.0,114.0,228.0,289548.0,39.371710,78.743421,31.0
Condado-Campiña,14381.0,73.0,124.0,156231.0,46.725682,79.369651,28.0
Huelva (capital),12715.0,59.0,96.0,143837.0,41.018653,66.742215,15.0
Moguer,2036.0,24.0,35.0,21867.0,109.754424,160.058536,13.0
Cartaya,2136.0,24.0,59.0,20083.0,119.504058,293.780810,9.0
Isla Cristina,3167.0,10.0,21.0,21393.0,46.744262,98.162950,5.0
Palos de la Frontera,1076.0,11.0,23.0,11742.0,93.680804,195.878045,3.0
San Juan del Puerto,768.0,5.0,7.0,9411.0,53.129317,74.381043,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,53.0,3.0,22.0,713.0,420.757363,3085.553997,1.0
Granado (El),46.0,1.0,6.0,532.0,187.969925,1127.819549,0.0
Cartaya,2136.0,24.0,59.0,20083.0,119.504058,293.780810,9.0
San Bartolomé de la Torre,388.0,7.0,10.0,3761.0,186.120713,265.886732,2.0
Minas de Riotinto,188.0,5.0,8.0,3812.0,131.164743,209.863589,1.0
Palos de la Frontera,1076.0,11.0,23.0,11742.0,93.680804,195.878045,3.0
Moguer,2036.0,24.0,35.0,21867.0,109.754424,160.058536,13.0
Isla Cristina,3167.0,10.0,21.0,21393.0,46.744262,98.162950,5.0
Santa Bárbara de Casa,47.0,1.0,1.0,1043.0,95.877277,95.877277,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Campofrío,53.0,3.0,22.0,713.0,420.757363,3085.553997,1.0,0.136364
Granado (El),46.0,1.0,6.0,532.0,187.969925,1127.819549,0.0,0.166667
Lepe,3047.0,3.0,13.0,27880.0,10.760402,46.628407,0.0,0.230769
Sierra de Huelva-Andévalo Central,4095.0,11.0,38.0,67391.0,16.322654,56.387351,2.0,0.289474
Nerva,254.0,1.0,3.0,5169.0,19.346102,58.038305,0.0,0.333333
Bollullos Par del Condado,1298.0,1.0,3.0,14387.0,6.950719,20.852158,0.0,0.333333
Palma del Condado (La),1436.0,2.0,6.0,10801.0,18.516804,55.550412,1.0,0.333333
Bonares,467.0,1.0,3.0,6060.0,16.501650,49.504950,0.0,0.333333
Ayamonte,2112.0,6.0,17.0,21104.0,28.430629,80.553450,1.0,0.352941
